In [6]:
import pandas as pd
import numpy as np
import pickle
from scipy import stats

In [20]:
def getCorrelation(groupDrugs, pathDrug, GEX):
    drug_df = pd.read_csv(pathDrug, index_col=0)
    temp = []
    for n in drug_df.columns:
        if 'PCC' in n and 'Graph' in n:
            temp.append(True)
        else:
            temp.append(False)
    Graph_PCC = drug_df.loc[:,temp]
    Graph_PCC = Graph_PCC.sort_values(by=Graph_PCC.columns.to_list(), ascending=False)
    IC50 = pd.read_csv('/data2/project/drugResponse/GDSC_DATASET_S1-S12/Table_S6_GDSC_Drug_response_IC50.csv')
    drug_list = IC50.columns.to_list()
#     top10_drugs = []
#     for i in Graph_PCC.index[:10]:
#         top10_drugs.append(drug_list[i])
#     drug_list = list(set(top10_drugs) | set(groupDrugs))
    drug_list = groupDrugs
    print(drug_list)
    res = pd.DataFrame()
    for index1, value1 in IC50[drug_list].iteritems():
        temp = []
        for index2, value2 in GEX.iteritems():
            temp.append(stats.pearsonr(value1[~np.isnan(value1)].values, value2[~np.isnan(value1)].values))
        res = pd.concat([res, pd.DataFrame(temp)], axis=1)
    res = res.set_index([GEX.columns])
    columns = []
    for n in drug_list:
        columns.append(n + '_correlation')
        columns.append(n + '_p-value')
    res.columns = columns
    return res

In [7]:
path1 = 'result/210126/L1000+AUC/1.csv'
GEX = pd.read_csv('GDSC+STRING/L1000/groupEXP.csv', index_col=0)
with open('result/PATHWAY_DRUG.pickle', 'rb') as f:
    data = pickle.load(f)

In [21]:
df = getCorrelation(data['Hormone-related'], path1, GEX)

['Bicalutamide', 'Tamoxifen']


In [23]:
df.to_csv('Hormone-related Result.csv')